In [3]:
import numpy as np
import pandas as pd
import time
import memory_profiler
%load_ext memory_profiler
import logging 
mpl_logger = logging.getLogger('matplotlib') 
mpl_logger.setLevel(logging.WARNING) 
import NaiveDE
import SpatialDE


def get_coords(index):
    coords = pd.DataFrame(index=index)
    coords['x'] = index.str.split('x').str.get(0).map(float)
    coords['y'] = index.str.split('x').str.get(1).map(float)
    return coords


In [4]:
nc=[]
tt =[]
import time
import warnings
warnings.filterwarnings('ignore')


n_cell=[100,250,500,1000,1500,5000,10000,15000,50000]
for i in n_cell:
    ff='../../../data/Simulation_data/1M_cells_100genes_counts_0.csv'
    df_0 = pd.read_csv(ff,sep=',', index_col = 0, iterator=True)

    df=df_0.get_chunk(i)
    df = df.T[df.sum(0) >= 3].T   
   
    sample_info = get_coords(df.index)
    sample_info['total_counts'] = df.sum(1)
    sample_info = sample_info.query('total_counts > 10')  # Remove empty features
    

    df = df.loc[sample_info.index]     # Align count matrix with metadata table
    X = sample_info[['x', 'y']]

# Convert data to log-scale, and account for depth
    dfm = NaiveDE.stabilize(df.T).T
    res = NaiveDE.regress_out(sample_info, dfm.T, 'np.log(total_counts)').T

# Add total_count as pseudogene for reference
    res['log_total_count'] = np.log(sample_info['total_counts'])

# Perform Spatial DE test with default settings
    start=time.time()
    %memit results = SpatialDE.run(X, res)
    finish=time.time()
    t=finish-start                # 15706 
    
    nc.append(int(i))    
    tt.append(t/60)




peak memory: 199.74 MiB, increment: 26.76 MiB




peak memory: 207.69 MiB, increment: 7.07 MiB




peak memory: 231.10 MiB, increment: 28.66 MiB




peak memory: 322.93 MiB, increment: 115.89 MiB




peak memory: 474.71 MiB, increment: 257.57 MiB




peak memory: 2913.32 MiB, increment: 2681.34 MiB
